금융위원회 보도자료에서 검색조건 사업보고서인 자료 다운로드

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

c:\Users\Master\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
driver = webdriver.Chrome()

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.fsc.go.kr/no010101')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [7]:
# 검색창 요소 찾기
search_box = driver.find_element(By.ID, 'srchText')

# 검색어 입력
search_box.send_keys('사업보고서')

# 검색 실행 (엔터 키 입력)
search_box.send_keys(Keys.RETURN)

# 결과 페이지가 로드될 때까지 대기 (필요시 조정)
time.sleep(5)

In [8]:
# 다운로드한 파일들의 URL을 저장할 집합
downloaded_files = set()

# 페이지 탐색 변수
cur_page = 1

while True:
    try:
        # 파일 리스트 요소 찾기
        file_list_divs = driver.find_elements(By.XPATH, '//div[@class="file-list"]')

        # PDF 파일 링크 필터링 및 다운로드
        pdf_download_links = []
        for file_div in file_list_divs:
            file_name_span = file_div.find_element(By.XPATH, './/span[@class="name"]')
            if 'pdf' in file_name_span.text.lower():
                download_link = file_div.find_element(By.XPATH, './/span[@class="ico download"]/a')
                download_url = download_link.get_attribute('href')
                if download_url not in downloaded_files:
                    pdf_download_links.append(download_link)
                    downloaded_files.add(download_url)

        # 더 이상 다운로드할 PDF 링크가 없으면 다음 페이지로 이동
        if not pdf_download_links:
            print(f"Page {cur_page}: 모든 PDF 파일을 다운로드했습니다.")

            # 다음 페이지로 이동
            cur_page += 1
            next_page_xpath = f'//a[@title="{cur_page} Page"]'
            next_page_link = driver.find_elements(By.XPATH, next_page_xpath)
            
            if next_page_link:
                next_page_link[0].click()
                time.sleep(5)  # 다음 페이지 로드 대기
                continue
            else:
                print("더 이상 페이지가 없습니다. 작업을 종료합니다.")
                break

        # PDF 다운로드 링크 클릭
        for link in pdf_download_links:
            link.click()
            time.sleep(2)  # 파일 다운로드 시간을 고려해 대기 시간 조정

    except Exception as e:
        print(f"오류 발생: {e}")
        break

Page 1: 모든 PDF 파일을 다운로드했습니다.
Page 2: 모든 PDF 파일을 다운로드했습니다.
Page 3: 모든 PDF 파일을 다운로드했습니다.
Page 4: 모든 PDF 파일을 다운로드했습니다.
Page 5: 모든 PDF 파일을 다운로드했습니다.
Page 6: 모든 PDF 파일을 다운로드했습니다.
Page 7: 모든 PDF 파일을 다운로드했습니다.
Page 8: 모든 PDF 파일을 다운로드했습니다.
Page 9: 모든 PDF 파일을 다운로드했습니다.
Page 10: 모든 PDF 파일을 다운로드했습니다.
더 이상 페이지가 없습니다. 작업을 종료합니다.


In [9]:
# 브라우저 닫기
driver.quit()

다운로드한 pdf파일에서 파이썬을 통해 자동화를 사용해서 기업명, 분식방법 추출

2. 구글에서 분식 관련 단어 검색으로 기사 크롤링 -> 기사 안에서 기업명 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

c:\Users\Master\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [25]:
driver = webdriver.Chrome()

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.google.co.kr/')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [26]:
# 검색창을 찾습니다.
search_box = driver.find_element(By.NAME, 'q')

# 검색어를 입력합니다.
search_box.send_keys('분식회계')

# 엔터 키를 눌러 검색을 실행합니다.
search_box.send_keys(Keys.RETURN)

# 잠시 대기합니다. (검색 결과 페이지가 로딩될 시간을 줍니다.)
time.sleep(3)

In [18]:
# 뉴스 아이콘을 클릭합니다.
news_icon = driver.find_element(By.XPATH, '//div[@jsname="bVqjv" and contains(text(), "뉴스")]')
news_icon.click()

# 크롤링할 데이터를 저장할 리스트 초기화
titles = []
urls = []

# 크롤링 수행
while True:
    try:
        # 뉴스 기사 제목과 URL 추출
        articles = driver.find_elements(By.XPATH, '//a[@jsname="YKoRaf"]')
        for article in articles:
            title_element = article.find_element(By.XPATH, './/div[@role="heading"]')
            title = title_element.text
            url = article.get_attribute('href')
            titles.append(title)
            urls.append(url)

        # 다음 페이지로 이동
        next_button = driver.find_elements(By.XPATH, '//a[@aria-label="다음 페이지"]')
        if next_button:
            next_button[0].click()
            time.sleep(3)  # 페이지 로드 대기
        else:
            break  # 다음 페이지 버튼이 없으면 종료
    except Exception as e:
        print(f"오류 발생: {e}")
        break

In [34]:
# 뉴스 아이콘을 클릭합니다.
news_icon = driver.find_element(By.XPATH, '//div[@jsname="bVqjv" and contains(text(), "뉴스")]')
news_icon.click()

# 크롤링할 데이터를 저장할 리스트 초기화
titles = []
urls = []

# 현재 페이지 번호
page_number = 1

# 크롤링 수행
while True:
    try:
        # 뉴스 기사 제목과 URL 추출
        articles = driver.find_elements(By.XPATH, '//a[@jsname="YKoRaf"]')
        for article in articles:
            title_element = article.find_element(By.XPATH, './/div[@role="heading"]')
            title = title_element.text
            url = article.get_attribute('href')
            titles.append(title)
            urls.append(url)
        
        # 다음 페이지로 이동
        page_number += 1
        next_button = driver.find_elements(By.XPATH, f'//a[@aria-label="Page {page_number}"]')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(3)  # 페이지 로드 대기
        else:
            break  # 다음 페이지 버튼이 없으면 종료
    except Exception as e:
        print(f"오류 발생: {e}")
        break

In [35]:
# 데이터프레임 생성
df = pd.DataFrame({'Title': titles, 'URL': urls})

In [36]:
df

,Title,URL
0,‘분식회계 의혹’ 카카오모빌리티 운명은?…제재 결론 늦어진다,https://www.khan.co.kr/economy/industry-trade/...
1,한때 잘나갔던 기업들의 회계 거짓말,https://www.taxwatch.co.kr/article/tax/2024/07...
2,구속 기로 카카오 김범수…'분식회계' 모빌리티 제재 임박,https://biz.sbs.co.kr/article/20000182200
3,"증선위, 코넥스 상장사 씨앗 검찰 고발…분식회계·횡령 은폐 등",https://m.dnews.co.kr/m_home/view.jsp?idxno=20...
4,"""2000억 날리고 직원도 떠났다""…'100조' 분식회계 후폭풍",https://www.hankyung.com/article/202407054056i
...,...,...
295,"서울중앙지검, '분식회계 혐의' 대우산업개발 압수수색",https://www.yna.co.kr/view/PYH20230413080000013
296,"中 러킨커피, 점포 1만점 목전...""분식회계 오명 털고 성장구가""",https://www.newspim.com/news/view/20230505000067
297,"[로펌의기술](84) ‘허위 공시’ 대한전선, 주주 손해배상 금액 다시 따져야......",https://biz.chosun.com/topics/law_firm/2022/10...
298,"대법 ""'분식회계' 대한전선·안진회계, 주주들에 손해배상하라""",https://www.newspim.com/news/view/20221011000257


In [38]:
#df.to_csv('분식기업크롤링.csv', encoding='utf-8-sig', index = False)

In [39]:
driver.quit()